In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import NMT
from NMT.utils import get_dataloader


In [ ]:
#training params
num_epochs = 1
#learning_rate = 1
#optimizer = "SGD"
#Next options are outdated in favour of optimizer schedulers?
#max_grad_norm
#learning_rate_decay
#start_decay_at

In [ ]:
#TODO change tokenizer tokens (adapt constatns to IDS?)
# bos_token = constants.BOS_WORD, eos_token, unk_token, sep_token, pad_token, cls_token, mask_token
tokenizer_en = AutoTokenizer.from_pretrained('distilbert-base-uncased')
# https://huggingface.co/dbmdz/bert-base-german-cased
tokenizer_de = AutoTokenizer.from_pretrained("dbmdz/bert-base-german-cased")

In [ ]:
#for num_epoch in num_epochs:
    #input_lang, output_lang, train_dataloader = get_dataloader(batch_size)
train_dataloader = get_dataloader(split="train", input_tokenizer=tokenizer_en, target_tokenizer=tokenizer_de, batch_size=batch_size)
test_dataloader = get_dataloader(split="test", input_tokenizer=tokenizer_en, target_tokenizer=tokenizer_de, batch_size=batch_size)

In [ ]:
from NMT import models
#num_layers,
# bidirectional,
# dropout,
#'-rnn_size', default=600, 
#'-word_vec_dim', default=400, # ONMT opt.word_vec_size
# dict_size # opt.data -> dict -> dict.size() 
#Parameters
rnn_size = 100
word_vec_dim = 50
lstm_layers = 2
dropout = 0.2
batch_size = 32

encoder = models.Encoder(num_layers=lstm_layers, bidirectional=True, dropout=dropout, rnn_size=rnn_size, word_vec_dim=word_vec_dim, dict_size=len(tokenizer_en), padding = tokenizer_en.convert_tokens_to_ids(tokenizer_en.pad_token))
decoder = models.Decoder(num_layers=lstm_layers, bidirectional=False, dropout=dropout, rnn_size=rnn_size, word_vec_dim=word_vec_dim, dict_size=len(tokenizer_de), padding = tokenizer_en.convert_tokens_to_ids(tokenizer_en.pad_token))

In [ ]:
nmtModel = models.NMTModel(encoder,decoder)

In [ ]:
from NMT.utils import train

train(train_dataloader, encoder, decoder, 1, print_every=5, plot_every=5)#learning rate # TODO add other optimizers

encoder.eval()
decoder.eval()